# L-9-2: Inverse Classroom: It’s not working! Help!

These exercises will give you some debugging experience on problems typically found when doing machine learning in practice.

**Outline**

0. General Set-up
1. Debugging A Bad Training Set-up
2. Image Segmentation with DICE Loss
3. Fixing the Data-Processing Pipeline
4. Test Performance is Too Good!

## 0. General Set-up

Here we provide general code set-up: package requirements, train-loaders, etc.

In [ ]:
## Some general imports we may need:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dataset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import time
import struct

Make sure GPU is enabled: In Colab, at the top, 

click `Runtime` -> `Change runtime type` -> `Hardware Accelerator` -> `GPU`

In [ ]:
gpu_boole = torch.cuda.is_available()

##2. Image Segmentation with DICE Loss

* Like in L-9-1, you are given a training loop for MNIST.
* However, in this case you will be performing image segmentation on the dataset with added noise.
* You will be introduced to the DICE loss and will need to implement such that it works with pytorch and the training loop.
* **Deliverables:**
    * Implement DICE loss such that the network trains with no errors thrown.
    * Describe what part of the formulation needed to be adjusted and for what reason.
    * Achieve a specific performance benchmark.

### 2.1 What is image segmentation?

* Image segmentation tasks consist of categorizing different parts of an image.
* Typically this is done pixel-wise.
* Pixel-wise segmentation is equilavent to classifying each pixel.
* Below is an example of image segmentation: (Taken from [this blog](https://nanonets.com/blog/how-to-do-semantic-segmentation-using-deep-learning/).)
![](https://cdn-images-1.medium.com/max/900/1*MQCvfEbbA44fiZk5GoDvhA.png)
* Each color represents a different category of the image.
* In this case, the neural network classified each pixel as one category (smoothing can also be applied afterwards).

### 2.2 What is Dice Loss?

* One popular loss function for image segmentation is the Dice loss.
* This originates from the Sørensen–Dice coefficient, a metric of similarity between two sets.
* For discrete sets $A$ and $B$, the Dice coefficient is defined as:
$$DSC = \frac{2| A \cap B | }{|A| + |B|}$$
* In other words, it is the proportion of common elements in $A$ and $B$. For DSC of 0, there is no overlap, for DSC of 1, $A = B$.

### 2.3 Applying Dice Loss to Machine Learning
* How can the Dice coefficient be adapted for a Machine Learning task?
* For binary segmentation (segmenting the image into two regions), for an image with $d$ pixels, the network output $f(x) \in [0,1]^d$, where $f(x)_k$ corresponds to the probability that the $k$-th pixel belongs to class 1, and $1- f(x)_k$ corresponds to the probability that the $k$-th pixel belongs to class 2.
* For implementation purposes, consider ground-truth $y = [0,0,0,1,1,1,0,0]$ and prediction $\hat{y} = [0.03,0.12,0.01,0.98,0.97,0.99,0.01,0.2]$.
* First, let's translate $| y \cap \hat{y}|$ into an arithmetic operation (which would work with auto-differentiation).

In [ ]:
y = np.array([0,0,0,1,1,1,0,0])
y_hat = np.array([0.03,0.12,0.01,0.98,0.97,0.99,0.01,0.2])

## Define similarity method (numerator only of DSC) here:
def DSC_num(A,B):
  ## [Your Code here]
  return

print(DSC_num(y, y_hat))

* Does your `DSC_num` function capture perfect overlap (i.e. returns 1 when A = B) and perfect non-overlap (i.e. reutrns 0 when $|A \cap B| = 0$)?
* Next, let's translate $|A|$ into an arithmetic operation as well and then define our DSC loss:

In [ ]:
## Define sum (denominator only of DSC) method here:
def DSC_den(A,B):
  ## [Your Code here]
  return

print(DSC_den(y, y_hat))

def DSC(A,B):  
  return DSC_num(A,B) / DSC_den(A,B)

print(DSC(y, y_hat))

* If you haven't already, define these functions such that they work with pytorch tensors:


In [ ]:
y = torch.from_numpy(np.array([0,0,0,1,1,1,0,0])).float()
y_hat = torch.from_numpy(np.array([0.03,0.12,0.01,0.98,0.97,0.99,0.01,0.2])).float()

print(DSC_num(y, y_hat))
print(DSC_den(y, y_hat))
print(DSC(y, y_hat))

*   You will also need to define these functions such that they work with minibatches of shape (batch_size, 784). Below, we give an example of a batch size of 4.
*   Your `DSC_batch` function will need to return a scalar. To do this, return the mean of the DSC loss for each sample.

In [ ]:
A = [0,0,0,1,1,1,0,0]
B = [0.03,0.12,0.01,0.98,0.97,0.99,0.01,0.2]
y = torch.from_numpy(np.array([A,A,A,A])).float()
y_hat = torch.from_numpy(np.array([B,B,B,B])).float()
print("Data shapes:",y.shape, y_hat.shape)

def DSC_num_batch(A,B):
  ## [Your Code here]
  return

def DSC_den_batch(A,B):
  ## [Your Code here]
  return

def DSC_batch(A,B):
  return (DSC_num_batch(A,B) / DSC_den_batch(A,B)).mean()

def DSC_loss(A,B):
  return 1 + -1.0*DSC_batch(A,B)

print("DSC function outputs:")
print(DSC_num_batch(y, y_hat))
print(DSC_den_batch(y, y_hat))
print(DSC_batch(y, y_hat))


### 2.4 Using Dice With Noisy MNIST Segmentation

Now, we are ready to test our Dice loss on binary segmentation of MNIST with noise.

**Defining the model and optimizer:**
We define the model and optimizer here.

In [ ]:
## Defining the model:
class Net(nn.Module):
  def __init__(self, input_size, width, num_classes):
    super(Net, self).__init__()

    ##feedfoward layers:
    self.ff1 = nn.Linear(input_size, width) #input

    self.ff2 = nn.Linear(width, width) #hidden layers
    self.ff3 = nn.Linear(width, width)

    self.ff_out = nn.Linear(width, num_classes) #logit layer     

    ##activations:
    self.relu = nn.ReLU()
    self.sm = nn.Softmax()
    self.sigmoid = nn.Sigmoid()
                
  def forward(self, input_data):
    out = self.relu(self.ff1(input_data)) 
    out = self.relu(self.ff2(out)) 
    out = self.relu(self.ff3(out))
    out = self.ff_out(out)
    return self.sigmoid(out) #returns probabilities for each pixel

net = Net(input_size = 784, width = 500, num_classes = 784)
if gpu_boole:
  net = net.cuda()

optimizer = torch.optim.SGD(net.parameters(), lr = 0.05)
loss_metric = DSC_loss

**Data loading:** In L-9-1, we made use of the `torchvision` package for data-loading and preprocessing. We abstracted away some preprocessing steps. Here, we are giving a more granular implementation that you may have to adjust in various ways. This is more akin to what you will see in practice with other datasets.


In [ ]:
#Downloading and unzipping MNIST data files:
!curl -O http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
!curl -O http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
!curl -O http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
!curl -O http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
!gunzip t*-ubyte.gz -f

##Loading files into numpy arrays and then torch Tensorsz:
def read_idx(filename, boole=0):
    with open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        if boole:
          return np.fromstring(f.read(), dtype=np.uint8).reshape(shape).astype(np.float32)*10     
        else:
          return np.fromstring(f.read(), dtype=np.uint8).reshape(shape)

xtrain = read_idx('train-images-idx3-ubyte', 1)
xtest = read_idx('t10k-images-idx3-ubyte', 1)

xtrain = torch.Tensor(xtrain)
xtest = torch.Tensor(xtest)

##normalizing between 0 and 1:
xtrain -= xtrain.min()
xtest -= xtest.min()
xtrain /= xtrain.max()
xtest /= xtest.max()

Of note, here, is that we use the original MNIST dataset for our ground-truth values. Given the simplicity of MNIST, simple rounding gives us close to the ground-truth for binary segmentation. In practice, with more complex datasets, there are more detailed images where simple rounding does not work, and manually labeled ground truth values must be used for training.

In [ ]:
ytrain = torch.round(xtrain)
ytest = torch.round(xtest)

**Adding noise:** Here, we add Gaussian noise with mean of 0.01 and standard deviation of 0.1 to a single MNIST image for illustration. This is to make the task harder and not too simplistic for this assignment. We will incorporate this noise adding into the training loop.

In [ ]:
# Image with noise:
print("Image with noise:")
plt.imshow((xtrain[0] + xtrain[0].data.new(xtrain[0].size()).normal_(0.01, 0.1)).cpu().data.numpy(), cmap = "gray")
plt.show()

# Ground truth:
print("Image segmentation ground-truth:")
plt.imshow(ytrain[0].cpu().data.numpy(), cmap = "gray")
plt.show()

# adding noise to all images:
for i in range(xtrain.shape[0]):
  xtrain[i] += xtrain[i].data.new(xtrain[i].size()).normal_(0.01, 0.1)
for i in range(xtest.shape[0]):
  xtest[i] += xtest[i].data.new(xtest[i].size()).normal_(0.01, 0.1)

# dataloaders:
train = torch.utils.data.TensorDataset(xtrain, ytrain)
test = torch.utils.data.TensorDataset(xtest, ytest)

train_loader = torch.utils.data.DataLoader(train, batch_size=128)
test_loader = torch.utils.data.DataLoader(test, batch_size=128, shuffle=False)

**Defining training and test loss functions:** These functions will be useful in our training loop to view are training and test loss at each epoch.

In [ ]:
def train_eval(verbose = 1):
    loss_sum = 0
    total = 0
    for images, labels in train_loader:
        if gpu_boole:
            images, labels = images.cuda(), labels.cuda()
        images = images.view(-1, 28*28)
        labels = labels.view(-1, 28*28)
        outputs = net(images)
        loss_sum += loss_metric(outputs,labels)

        total += labels.size(0)
        
    if verbose:
        print('Train loss: %f' % (loss_sum.cpu().data.numpy().item() / total))

    return loss_sum.cpu().data.numpy().item() / total
    
def test_eval(verbose = 1):
    loss_sum = 0
    total = 0
    for images, labels in test_loader:
        if gpu_boole:
            images, labels = images.cuda(), labels.cuda()
        images = images.view(-1, 28*28)
        labels = labels.view(-1, 28*28)
        outputs = net(images)
        loss_sum += loss_metric(outputs,labels)

        total += labels.size(0)
        
    if verbose:
        print('Test loss: %f' % (loss_sum.cpu().data.numpy().item() / total))

    return loss_sum.cpu().data.numpy().item() / total


**Training loop:** We are finally ready to begin training. Your own `DSC_pytorch` is called for computing the loss. It's possible you may need to debug this in the loop. We given an output of your segmented image and compare it to ground truth below.

**IMPORTANT NOTE:** For re-running this code cell, if you encounter nan loss, you will need to reinstantiate your model and optimizer by re-running the "Defining the model and optimizer:" code cell above.

If you modify your DSC_loss definitions, you will also need to re-run the "Defining the model and optimizer:" code cell above.

In [ ]:
#re-initializing network weights:
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform(m.weight.data)

weights_init(net)

#number of epochs to train for:
epochs = 10

#defining batch train loss recording arrays for later visualization/plotting:
loss_batch_store = []

print("Starting Training")
#training loop:
for epoch in range(epochs):
  time1 = time.time() #timekeeping

  for i, (x,y) in enumerate(train_loader):

    if gpu_boole:
      x = x.cuda()
      y = y.cuda()

    x = x.view(x.shape[0],-1)
    y = y.view(y.shape[0],-1)

    #loss calculation and gradient update:

    if i > 0 or epoch > 0:
      optimizer.zero_grad()
    outputs = net.forward(x)
    loss = loss_metric(outputs,y)
    loss.backward()

    if i > 0 or epoch > 0:
      loss_batch_store.append(loss.cpu().data.numpy().item())
                  
    ##performing update:
    optimizer.step()

  print("Epoch",epoch+1,':')
  train_loss = train_eval()
  test_loss = test_eval()

  time2 = time.time() #timekeeping
  print('Elapsed time for epoch:',time2 - time1,'s')
  print('ETA of completion:',(time2 - time1)*(epochs - epoch - 1)/60,'minutes')
  print()

## Plotting batch-wise train loss curve:
plt.plot(loss_batch_store, '-o', label = 'train_loss', color = 'blue')
plt.xlabel('Minibatch Number')
plt.ylabel('Sample-wise Loss At Last minibatch')
plt.legend()
plt.show()

## Plotting sample xtest segmentation and prediction:
print("Noisy xtest image:")
plt.imshow(xtest[0].cpu().data.numpy(), cmap = "gray")
plt.show()

print("Ground-truth:")
plt.imshow(ytest[0].cpu().data.numpy(), cmap = "gray")
plt.show()

print("Prediction:")
if gpu_boole:
  out = net.forward(xtest[0].cuda().view(1,28*28)).view(28,28).cpu().data.numpy()
else:
  out = net.forward(xtest[0].view(1,28*28)).view(28,28).cpu().data.numpy()
plt.imshow(out, cmap = "gray")
plt.show()

**Unit testing:**

Okay, does your final output look correct? Even if you network is segmenting correctly, it may not be doing so optimally. You should unit test your DSC_batch function to make sure it is returning the correct value.

Hand-calculate the DSC loss for two specific inputs, and make sure your DSC_batch function is returning the correct value:

In [ ]:
#Unit testing DSC_batch:

#TO-DO: Insert your specific testing vectors here:
y1 = []
y1_hat = []

#O-DO: Insert your specific testing vectors here:
y2 = []
y2_hat = []

y = torch.from_numpy(np.array([y1,y2])).float()
y_hat = torch.from_numpy(np.array([y1_hat,y2_hat])).float()
print("Data shapes:",y.shape, y_hat.shape)

print("DSC function outputs:")
print(DSC_num_batch(y, y_hat))
print(DSC_den_batch(y, y_hat))
print(DSC_batch(y, y_hat))


**Questions:**
* How did you implement the overlap metric and the sum metric? What adjustments were needed (if any) to made to make it amenable to pytorch?

[Your text here.]

* Why might the Dice loss be advantageous compared to cross entropy? 

[Your text here.]

* Are there cases where cross entropy might be advantageous compared to the Dice loss? 

[Your text here.]

* Did you unit test your Dice loss? Did it help you catch errors or what it already correctly specified? 

[Your text here.]